Code accompanying the manuscript: "Quantification of similarity and physical awareness of microstructures generated via Generative Models" by Sanket Thakre, Vir Karan, Anand Krishna Kanjarla.

The following code evaluates the ROM predictions (physical awareness of GAN microstructures) and latent space studies

## Imports

In [ ]:
import os
import numpy as np
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from statistics import variance 
from matplotlib.ticker import FuncFormatter, MaxNLocator
from google.colab import files
from sklearn.metrics import r2_score
import matplotlib.font_manager

Import auto-correlations data 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%%time
# Import the MATLAB generated correlations data file (256 x 256 RVE)
corr_final = pd.read_csv('/content/gdrive/My Drive/Work from home/Vir files/Damage prediction/corr_DP.csv', header = None)
a = corr_final.values

## Dimensionality reduction

We will use PCA for reducing dimensionality of the data and for generating ROM

In [ ]:
ID = 3
pca = PCA(n_components = ID).fit(anew)
anew_pca = pca.transform(anew)

var = np.cumsum(pca.explained_variance_ratio_)

In [ ]:
# For value in range +1 to the no of components 
fig = plt.figure(figsize=plt.figaspect(1)*1.5)
plt.plot(range(1,ID+1),var,'-o',label='Explained variance ratio')

plt.title('Scree plot for PCA', fontsize = 18)
plt.ylabel('Explained variance ratio',  fontsize = 16)
plt.xlabel('Components',  fontsize = 16)
plt.axhline(y = 0.95, color='k', linestyle='--', label = '95% Explained Variance')
plt.axhline(y = 0.90, color='r', linestyle='--', label = '90% Explained Variance')
plt.legend(loc='lower right', fontsize = 11)
plt.tick_params(axis='both', labelsize=14)
plt.ylim(0.5,1)
plt.grid(True)
plt.show()

print('With the ',ID,' components, we can retain',sum(pca.explained_variance_ratio_)*100,'percent data')

In [ ]:
all_stack_pca = anew_pca
hfont = {'fontname':'Sans'}
fig = plt.figure(figsize=plt.figaspect(1)*2)
plt.axis(aspect='equal')
ax = plt.axes(projection='3d')
zdata = all_stack_pca[0:50,2]
xdata = all_stack_pca[0:50,0]
ydata = all_stack_pca[0:50,1]
ax.scatter3D(xdata, ydata, zdata, c='red', s=80, marker='o', label='Class 1');
zdata = all_stack_pca[50:100,2]
xdata = all_stack_pca[50:100,0]
ydata = all_stack_pca[50:100,1]
ax.scatter3D(xdata, ydata, zdata, c='blue', s=80,marker='o',label='Class 2');
zdata = all_stack_pca[100:150,2]
xdata = all_stack_pca[100:150,0]
ydata = all_stack_pca[100:150,1]
ax.scatter3D(xdata, ydata, zdata, c='green', s=80, marker='o',label='Class 3');
zdata = all_stack_pca[150:200,2]
xdata = all_stack_pca[150:200,0]
ydata = all_stack_pca[150:200,1]
ax.scatter3D(xdata, ydata, zdata, c='yellow', s=80, marker='o',label='Class 4');
zdata = all_stack_pca[200:250,2]
xdata = all_stack_pca[200:250,0]
ydata = all_stack_pca[200:250,1]
ax.scatter3D(xdata, ydata, zdata, c='black', s=80, marker='o',label='Class 5');
zdata = all_stack_pca[250:300,2]
xdata = all_stack_pca[250:300,0]
ydata = all_stack_pca[250:300,1]
ax.scatter3D(xdata, ydata, zdata, c='orange', s=80, marker='o',label='Class 6');
plt.legend(fontsize = 11)

ax.tick_params(axis='both', labelsize=14)
ax.set_xlabel('PC1', fontname='Helvetica', fontsize = 18)
ax.set_ylabel('PC2', fontsize = 18)
ax.set_zlabel('PC3', fontsize = 18)
plt.title('3D PC representation ', fontname="Brush Script MT", fontsize = 18, )
plt.show()

 ## Input data preparation

Input data (900 x 4): First three columns will be three times 300 x 3 PC components and 4th column will be work hardening coeffcients

In [ ]:
h1 = np.reshape(0.24 * np.ones(300),[300,1])
h2 = np.reshape(0.34 * np.ones(300),[300,1])
h3 = np.reshape(0.44 * np.ones(300),[300,1])
pc1 = np.hstack([anew_pca,h1])
pc2 = np.hstack([anew_pca,h2])
pc3 = np.hstack([anew_pca,h3])

inp_data = np.vstack([pc1,pc2,pc3])

In [ ]:
inp_data.shape

## ROM Generation import

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_absolute_percentage_error as mpe
from sklearn.metrics import r2_score as r2

In [ ]:
%%time
# Import the damage initiation stress data for all 900 RVEs (900 x 1)
dam_1 = pd.read_csv('/content/gdrive/My Drive/Work from home/Vir files/Damage prediction/Damage_0.24.csv', header = None)
dam_2 = pd.read_csv('/content/gdrive/My Drive/Work from home/Vir files/Damage prediction/Damage_0.34.csv', header = None)
dam_3 = pd.read_csv('/content/gdrive/My Drive/Work from home/Vir files/Damage prediction/Damage_0.44.csv', header = None)
y1 = dam_1.values
y2 = dam_2.values
y3 = dam_3.values

In [ ]:
y = np.vstack([y1,y2,y3])
y.shape

# Random forest model

In [ ]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# To check which microstructure in taken in random split
h1 = np.reshape(1 * np.ones(50),[50,1])
h2 = np.reshape(2 * np.ones(50),[50,1])
h3 = np.reshape(3 * np.ones(50),[50,1])
h4 = np.reshape(4 * np.ones(50),[50,1])
h5 = np.reshape(5 * np.ones(50),[50,1])
h6 = np.reshape(6 * np.ones(50),[50,1])
cls = np.vstack([h1,h2,h3,h4,h5,h6])
index = np.vstack([cls,cls,cls])
inp_data1 = np.hstack([index,inp_data])

In [ ]:
# Data split  
x_train, x_test, y_train, y_test = train_test_split(inp_data1,y,test_size=0.2,random_state=1)
class_train = x_train[:,1:5]; class_test = x_test[:,1:5];  out_train= y_train; out_test = y_test; 

In [ ]:
# Model fitting
dt=ensemble.RandomForestRegressor(n_estimators=20,max_depth=7,random_state=1)
dt.fit(class_train,out_train)
dt.score(class_test,out_test)

In [ ]:
dt.score(class_train,out_train)

0.9851901677190446

In [ ]:
# Cross validation
predicted = cross_val_predict(ensemble.RandomForestRegressor(n_estimators=20,max_depth=7,random_state=1), class_train,out_train, cv=10)

In [ ]:
# Model testing on metrics
pred_class_test = dt.predict(class_test)
r2(out_test,pred_class_test)

0.9704940282888259

In [ ]:
mse(out_test,pred_class_test)

537.4702915701525

In [ ]:
mae(out_test,pred_class_test)

18.30262340625969

In [ ]:
mpe(out_test,pred_class_test)

0.018827821099555116

In [ ]:
importances = dt.feature_importances_

forest_importances = pd.Series(importances, index=['PC1', 'PC2', 'PC3', 'WH'])
std = np.std([tree.feature_importances_ for tree in dt.estimators_], axis=0)

fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=std, ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()

In [ ]:
forest_importances

In [ ]:
from sklearn.tree import plot_tree

fig = plt.figure(figsize=(50, 50))
#fig = plt.figure
plot_tree(dt.estimators_[4], feature_names= ['PC1', 'PC2', 'PC3', 'WH'])


# Actual & GAN microstructure comparison


In [ ]:
%%time
# Import the damage initiation stress data for all 900 RVEs (900 x 1)
dam_m = pd.read_csv('/content/gdrive/My Drive/Work from home/Vir files/Damage prediction/corr_m.csv', header = None)
dam_g = pd.read_csv('/content/gdrive/My Drive/Work from home/Vir files/Damage prediction/corr_g.csv', header = None)
ind_1 = pd.read_csv('/content/gdrive/My Drive/Work from home/Vir files/Damage prediction/Index_max_0.24.csv', header = None)
ind_2 = pd.read_csv('/content/gdrive/My Drive/Work from home/Vir files/Damage prediction/Index_max_0.34.csv', header = None)
ind_3 = pd.read_csv('/content/gdrive/My Drive/Work from home/Vir files/Damage prediction/Index_max_0.44.csv', header = None)

dam_g_lat = pd.read_csv('/content/gdrive/My Drive/Work from home/Vir files/Damage prediction/corr_g_lat.csv', header = None)

x1 = dam_m.values
x2 = dam_g.values
x3 = dam_g_lat.values
y_all = np.vstack([ind_1.values,ind_2.values,ind_3.values])


In [ ]:
# Performing PCA on all at once
ID = 3
pca_m = PCA(n_components = ID).fit(x1)
anew_pca_m = pca_m.transform(x1)
pca_g = PCA(n_components = ID).fit(x2)
anew_pca_g = pca_g.transform(x2)

In [ ]:
print('With the ',ID,' components, we can retain',sum(pca_g.explained_variance_ratio_)*100,'percent data')

In [ ]:
print('With the ',ID,' components, we can retain',sum(pca_m.explained_variance_ratio_)*100,'percent data')

In [ ]:
term1 = np.vstack([anew_pca_m[0,:],anew_pca_m[6,:],anew_pca_m[12,:]])
term2 = np.vstack([anew_pca_m[1,:],anew_pca_m[7,:],anew_pca_m[13,:]])
term3 = np.vstack([anew_pca_m[2,:],anew_pca_m[8,:],anew_pca_m[14,:]])
term4 = np.vstack([anew_pca_m[3,:],anew_pca_m[9,:],anew_pca_m[15,:]])
term5 = np.vstack([anew_pca_m[4,:],anew_pca_m[10,:],anew_pca_m[16,:]])
term6 = np.vstack([anew_pca_m[5,:],anew_pca_m[11,:],anew_pca_m[17,:]])
class_wise_m = np.vstack([term1,term2,term3,term4,term5,term6,])
class_wise_m

In [ ]:
term1 = np.vstack([anew_pca_g[0,:],anew_pca_g[6,:],anew_pca_g[12,:]])
term2 = np.vstack([anew_pca_g[1,:],anew_pca_g[7,:],anew_pca_g[13,:]])
term3 = np.vstack([anew_pca_g[2,:],anew_pca_g[8,:],anew_pca_g[14,:]])
term4 = np.vstack([anew_pca_g[3,:],anew_pca_g[9,:],anew_pca_g[15,:]])
term5 = np.vstack([anew_pca_g[4,:],anew_pca_g[10,:],anew_pca_g[16,:]])
term6 = np.vstack([anew_pca_g[5,:],anew_pca_g[11,:],anew_pca_g[17,:]])
class_wise_g = np.vstack([term1,term2,term3,term4,term5,term6,])
class_wise_g

# GAN microstructure prediction using ROM

In [ ]:
# Input data generation
h_1 = np.reshape(0.24 * np.ones(6),[6,1])
h_2 = np.reshape(0.34 * np.ones(6),[6,1])
h_3 = np.reshape(0.44 * np.ones(6),[6,1])
inp_h = np.vstack([h_1,h_2,h_3])

In [ ]:
inp_g = np.hstack([anew_pca_g,inp_h])
inp_m = np.hstack([anew_pca_m,inp_h])

In [ ]:
# Predicting using random forest model
pred_g = dt.predict(inp_g)
print("Rsquare (class test): %f" %(dt.score(inp_g, y_all[:,1])))

In [ ]:
# Predicting using random forest model
pred_m = dt.predict(inp_m)
print("Rsquare (class test): %f" %(dt.score(inp_m, y_all[:,1])))

In [ ]:
# Hardening coefficients wise plot
fig = plt.figure(figsize=plt.figaspect(1)*1.5)
plt.plot(y_all[0:6,1],pred_g[0:6],'o',label='Hard 0.24')
plt.plot(y_all[6:13,1],pred_g[6:13],'o',label='Hard 0.34')
plt.plot(y_all[13:18,1],pred_g[13:18],'o',label='Hard 0.44')

#plt.title('QQ plot', fontsize = 18)
plt.ylabel('Prediction', fontsize = 16)
plt.xlabel('Actual', fontsize = 16)
#plt.axhline(y = 0.95, color='k', linestyle='--', label = '95% Explained Variance')

plt.plot([0,1300],[0,1300],color='k', linestyle='--',linewidth=0.5)
plt.legend(loc='lower right', fontsize = 11)
plt.tick_params(axis='both', labelsize=14)
plt.ylim(650,1300)
plt.xlim(650,1300)
plt.grid(True)
#ax.set_facecolor('white')
plt.show()

In [ ]:
# Class wise plot
fig = plt.figure(figsize=plt.figaspect(1)*1.5)
plt.plot(y_all[0,1],pred_g[0],'ob',label='Class 1')
plt.plot(y_all[6,1],pred_g[6],'ob')
plt.plot(y_all[12,1],pred_g[12],'ob')
plt.plot(y_all[1,1],pred_g[1],'or',label='Class 2')
plt.plot(y_all[7,1],pred_g[7],'or')
plt.plot(y_all[13,1],pred_g[13],'or')
plt.plot(y_all[2,1],pred_g[2],'oy',label='Class 3')
plt.plot(y_all[8,1],pred_g[8],'oy')
plt.plot(y_all[14,1],pred_g[14],'oy')
plt.plot(y_all[3,1],pred_g[3],'og',label='Class 4')
plt.plot(y_all[9,1],pred_g[9],'og')
plt.plot(y_all[15,1],pred_g[15],'og')
plt.plot(y_all[4,1],pred_g[4],'ok',label='Class 5')
plt.plot(y_all[10,1],pred_g[10],'ok')
plt.plot(y_all[16,1],pred_g[16],'ok')
plt.plot(y_all[5,1],pred_g[5],'oc',label='Class 6')
plt.plot(y_all[11,1],pred_g[11],'oc')
plt.plot(y_all[17,1],pred_g[17],'oc')

plt.plot([0,1300],[0,1300],color='k', linestyle='--',linewidth=0.5)
#plt.title('QQ plot', fontsize = 18)
plt.ylabel('Prediction', fontsize = 16)
plt.xlabel('Actual', fontsize = 16)


plt.legend(loc='lower right', fontsize = 11)
plt.tick_params(axis='both', labelsize=14)
plt.ylim(650,1300)
plt.xlim(650,1300)
plt.grid(True)

plt.show()

In [ ]:
r = np.empty(6)
for i in range (6):
  m_pre = np.vstack([pred_m[i],pred_m[i+6],pred_m[i+12]])
  g_pre = np.vstack([pred_g[i],pred_g[i+6],pred_g[i+12]])
  r[i] = r2_score(m_pre, g_pre)
r

# Latent space studies

In [ ]:
%%time
space = np.zeros(512)
lay = 1

for i in range(600):
  data = pd.read_csv('/content/gdrive/My Drive/Work from home/Vir files/Visualisation/sampled_latents/'+ str(i) +'.csv')
  lat = data.values
  dum = lat[:,lay]
  space = np.vstack([space,dum])

obs_lay = space[1:601,:]

CPU times: user 2.24 s, sys: 111 ms, total: 2.35 s
Wall time: 2.95 s


In [ ]:
obs_lay.shape

(300, 512)

In [ ]:
ID = 3
pca = PCA(n_components = ID).fit(obs_lay)
anew_pca = pca.transform(obs_lay)

In [ ]:
pca.explained_variance_ratio_

In [ ]:
print('With the ',ID,' components, we can retain',sum(pca.explained_variance_ratio_)*100,'percent data')

In [ ]:
all_stack_pca = anew_pca

fig = plt.figure(figsize=plt.figaspect(1)*2)
plt.axis(aspect='equal')
ax = plt.axes(projection='3d')
zdata = all_stack_pca[0:100,2]
xdata = all_stack_pca[0:100,0]
ydata = all_stack_pca[0:100,1]
ax.scatter3D(xdata, ydata, zdata, c='red', s=80, marker='o', label='Class 1');
zdata = all_stack_pca[100:200,2]
xdata = all_stack_pca[100:200,0]
ydata = all_stack_pca[100:200,1]
ax.scatter3D(xdata, ydata, zdata, c='blue', s=80,marker='o',label='Class 2');
zdata = all_stack_pca[200:300,2]
xdata = all_stack_pca[200:300,0]
ydata = all_stack_pca[200:300,1]
ax.scatter3D(xdata, ydata, zdata, c='green', s=80, marker='o',label='Class 3');
zdata = all_stack_pca[300:400,2]
xdata = all_stack_pca[300:400,0]
ydata = all_stack_pca[300:400,1]
ax.scatter3D(xdata, ydata, zdata, c='yellow', s=80, marker='o',label='Class 4');
zdata = all_stack_pca[400:500,2]
xdata = all_stack_pca[400:500,0]
ydata = all_stack_pca[400:500,1]
ax.scatter3D(xdata, ydata, zdata, c='black', s=80, marker='o',label='Class 5');
zdata = all_stack_pca[500:600,2]
xdata = all_stack_pca[500:600,0]
ydata = all_stack_pca[500:600,1]
ax.scatter3D(xdata, ydata, zdata, c='orange', s=80, marker='o',label='Class 6');
plt.legend(fontsize = 11)

ax.tick_params(axis='both', labelsize=14)
ax.set_xlabel('PC1', fontsize = 18)
ax.set_ylabel('PC2', fontsize = 18)
ax.set_zlabel('PC3', fontsize = 18)
plt.title('3D PC representation ', fontsize = 18, )
plt.show()